In [1]:
import warnings

import geopandas as gpd
import libpysal
import momepy
import osmnx
import os
import configparser
import numpy as np

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
# Define cities and their respective CRS
cities = {
    "Ariel": "EPSG:2039",
    "Acre, Israel": "EPSG:2039",
    "Harish": "EPSG:2039", 
    "Modiin": "EPSG:2039",   
    "Carmiel": "EPSG:2039", 
    "Holon":  "EPSG:2039",
    "Hadera": "EPSG:2039",
    "Bnei Brak": "EPSG:2039",
    "Herzliya": "EPSG:2039",
    "Rehovot": "EPSG:2039",
    "Haifa": "EPSG:2039",  # Replace with actual local CRS for each city
    "Tel Aviv": "EPSG:2039",
    "Jerusalem": "EPSG:2039",
    "Beer Sheva": "EPSG:2039"
}

In [3]:
# Read the configuration file
config = configparser.ConfigParser()
config_file = 'config.ini'

# Check if file exists
if not os.path.isfile(config_file):
    # Create a simple config file
    config['Paths'] = {
        'gdb_folder': '/path/to/your/gdb/folder',
        'output_dir': '/path/to/your/output/folder'
    }
    with open(config_file, 'w') as configfile:
        config.write(configfile)
    print(f"Created configuration file '{config_file}'.")

# Read the configuration file
config.read(config_file)
gdb_folder = config['Paths']['gdb_folder']
output_dir = config['Paths']['output_dir']

# Ensure the directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:

# Dictionaries to store data for each city
buildings_data = {}
streets_data = {}
tessellation_data = {}

# Loop through each city calculate or load buildings, streets and tessealtions
for city, local_crs in cities.items():
    print(f"Processing {city}")

    # File paths for the saved data
    buildings_file = os.path.join(output_dir, f"{city}_buildings.geojson")
    tessellation_file = os.path.join(output_dir, f"{city}_tessellation.geojson")
    streets_file = os.path.join(output_dir, f"{city}_streets.geojson")

    # Check if files already exist, if so, load them
    if os.path.exists(buildings_file) and os.path.exists(tessellation_file) and os.path.exists(streets_file):
        print(f"Loading saved data for {city}...")
        buildings = gpd.read_file(buildings_file)
        tessellation = gpd.read_file(tessellation_file)
        streets = gpd.read_file(streets_file)
    else:
        print(f"Fetching and processing data for {city}...")
        # Get buildings and street data
        buildings = osmnx.geometries.geometries_from_place(city, tags={'building': True})
        buildings = buildings[buildings.geom_type == "Polygon"].reset_index(drop=True)
        buildings = buildings[["geometry"]].to_crs(local_crs)
        buildings["uID"] = range(len(buildings))

        osm_graph = osmnx.graph_from_place(city, network_type='drive')
        osm_graph = osmnx.projection.project_graph(osm_graph, to_crs=local_crs)

        streets = osmnx.graph_to_gdfs(
            osm_graph,
            nodes=False,
            edges=True,
            node_geometry=False,
            fill_edge_geometry=True
        )
        streets = momepy.remove_false_nodes(streets)
        streets = streets[["geometry"]]
        streets["nID"] = range(len(streets))

        # Generate tessellation and perform metrics
        limit = momepy.buffered_limit(buildings, 100)
        tessellation = momepy.Tessellation(buildings, "uID", limit, verbose=False, segment=1)
        tessellation = tessellation.tessellation

        buildings = buildings.sjoin_nearest(streets, max_distance=1000, how="left")
        buildings = buildings.drop_duplicates("uID").drop(columns="index_right")
        tessellation = tessellation.merge(buildings[['uID', 'nID']], on='uID', how='left')

        # Save the data for future use
        print(f"Saving data for {city}...")
        buildings.to_file(buildings_file, driver="GeoJSON")
        tessellation.to_file(tessellation_file, driver="GeoJSON")
        streets.to_file(streets_file, driver="GeoJSON")
    
    # Store the data in dictionaries
    buildings_data[city] = buildings
    streets_data[city] = streets
    tessellation_data[city] = tessellation

Processing Ariel
Loading saved data for Ariel...
Processing Acre, Israel
Loading saved data for Acre, Israel...
Processing Harish
Loading saved data for Harish...
Processing Modiin
Loading saved data for Modiin...
Processing Carmiel
Loading saved data for Carmiel...
Processing Holon
Loading saved data for Holon...
Processing Hadera
Loading saved data for Hadera...
Processing Bnei Brak
Loading saved data for Bnei Brak...
Processing Herzliya
Loading saved data for Herzliya...
Processing Rehovot
Loading saved data for Rehovot...
Processing Haifa
Loading saved data for Haifa...
Processing Tel Aviv
Loading saved data for Tel Aviv...
Processing Jerusalem
Loading saved data for Jerusalem...
Processing Beer Sheva
Loading saved data for Beer Sheva...


neighbors distance Q1

In [5]:
# create output directories
for dirname in ["neighbor_distance_Q1", "neighbor_distance_Q3", "shared_walls_length", "covered_area"]:
    dir_to_make = os.path.join(output_dir, dirname)
    if not os.path.exists(dir_to_make):
        os.makedirs(dir_to_make)

In [6]:
output_df_dir = os.path.join(output_dir, "neighbor_distance_Q1") # Replace with your desired folder path

# Dictionary to store the DataFrame for each city
city_metric_dfs = {}

# Loop through each city to calculate metrics based on stored data
for city in buildings_data.keys():
    print(f"Calculating metrics for {city}...")

    # Retrieve the data for the city from the dictionaries
    buildings = buildings_data[city]
    streets = streets_data[city]
    tessellation = tessellation_data[city]  
    
    queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tessellation, ids="uID", silence_warnings=True)
    tessellation["neighbors"] = momepy.Neighbors(tessellation, queen_1, "uID", weighted=True, verbose=False).series
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        buildings["neighbor_distance"] = momepy.NeighborDistance(buildings, queen_1, "uID", verbose=False).series

    buildings["log_neighbor_distance"] = np.log(buildings["neighbor_distance"] + 1) / np.log(10)

    # Create a DataFrame containing the necessary metrics for this city
    city_metrics_df = buildings[["uID", "neighbor_distance", "log_neighbor_distance"]].copy()

    # Store the DataFrame in the dictionary
    city_metric_dfs[city] = city_metrics_df

    # Display the DataFrame for the current city
    print(f"\nMetrics DataFrame for {city}:")
    print(city_metric_dfs[city].head())

    # Define the output file path using city name and metric
    output_file = os.path.join(output_df_dir, f"{city}_neighbor_distance_Q1.csv")

    # Save the DataFrame as a CSV file
    city_metrics_df.to_csv(output_file, index=False)

    print(f"Saved metrics DataFrame for {city} to {output_file}")


Calculating metrics for Ariel...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/1836647266.py:16: FutureWarning: Class based API like `momepy.Neighbors` is deprecated. Replace it with `momepy.neighbors` to use functional API instead or pin momepy version <1.0. Class-based API will be removed in 1.0. 
  tessellation["neighbors"] = momepy.Neighbors(tessellation, queen_1, "uID", weighted=True, verbose=False).series



Metrics DataFrame for Ariel:
   uID  neighbor_distance  log_neighbor_distance
0    0          44.559869               1.658582
1    1          70.371920               1.853527
2    2          20.534444               1.333134
3    3          16.153598               1.234355
4    4          30.653729               1.500425
Saved metrics DataFrame for Ariel to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q1/Ariel_neighbor_distance_Q1.csv
Calculating metrics for Acre, Israel...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/1836647266.py:16: FutureWarning: Class based API like `momepy.Neighbors` is deprecated. Replace it with `momepy.neighbors` to use functional API instead or pin momepy version <1.0. Class-based API will be removed in 1.0. 
  tessellation["neighbors"] = momepy.Neighbors(tessellation, queen_1, "uID", weighted=True, verbose=False).series



Metrics DataFrame for Acre, Israel:
   uID  neighbor_distance  log_neighbor_distance
0    0          60.604928               1.789615
1    1          91.686225               1.967015
2    2           0.000000               0.000000
3    3           0.000000               0.000000
4    4           7.123223               0.909728
Saved metrics DataFrame for Acre, Israel to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q1/Acre, Israel_neighbor_distance_Q1.csv
Calculating metrics for Harish...

Metrics DataFrame for Harish:
   uID  neighbor_distance  log_neighbor_distance
0    0          60.255488               1.787145
1    1          68.789163               1.843788
2    2          29.984734               1.491148
3    3          38.641420               1.598149
4    4          50.426352               1.711186
Saved metrics DataFrame for Harish to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q1/Harish_neighbor_distance_Q1.csv
Calculating me

...


Metrics DataFrame for Tel Aviv:
   uID  neighbor_distance  log_neighbor_distance
0    0          47.506637               1.685801
1    1          67.795453               1.837560
2    2         119.657735               2.081555
3    3          32.044263               1.519096
4    4         144.011729               2.161403
Saved metrics DataFrame for Tel Aviv to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q1/Tel Aviv_neighbor_distance_Q1.csv
Calculating metrics for Jerusalem...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/1836647266.py:16: FutureWarning: Class based API like `momepy.Neighbors` is deprecated. Replace it with `momepy.neighbors` to use functional API instead or pin momepy version <1.0. Class-based API will be removed in 1.0. 
  tessellation["neighbors"] = momepy.Neighbors(tessellation, queen_1, "uID", weighted=True, verbose=False).series



Metrics DataFrame for Jerusalem:
   uID  neighbor_distance  log_neighbor_distance
0    0          41.893135               1.632388
1    1          20.728500               1.337030
2    2         122.810058               2.092756
3    3           3.291272               0.632586
4    4           2.412582               0.533083
Saved metrics DataFrame for Jerusalem to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q1/Jerusalem_neighbor_distance_Q1.csv
Calculating metrics for Beer Sheva...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/1836647266.py:16: FutureWarning: Class based API like `momepy.Neighbors` is deprecated. Replace it with `momepy.neighbors` to use functional API instead or pin momepy version <1.0. Class-based API will be removed in 1.0. 
  tessellation["neighbors"] = momepy.Neighbors(tessellation, queen_1, "uID", weighted=True, verbose=False).series



Metrics DataFrame for Beer Sheva:
   uID  neighbor_distance  log_neighbor_distance
0    0          69.386202               1.847488
1    1          14.689683               1.195614
2    2          44.410318               1.657155
3    3          32.720991               1.527900
4    4          23.660412               1.392000
Saved metrics DataFrame for Beer Sheva to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q1/Beer Sheva_neighbor_distance_Q1.csv


neighbors distance Q3

In [7]:
output_df_dir = os.path.join(output_dir, "neighbor_distance_Q3")

# Dictionary to store the DataFrame for each city
city_metric_dfs = {}

# Loop through each city to calculate metrics based on stored data
for city in buildings_data.keys():
    print(f"Calculating metrics for {city}...")

    # Retrieve the data for the city from the dictionaries
    buildings = buildings_data[city]
    streets = streets_data[city]
    tessellation = tessellation_data[city]  
    
    queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tessellation, ids="uID", silence_warnings=True)
    # Create a higher-order spatial weights matrix for tessellation (considering neighbors up to 3rd order)
    queen_3 = momepy.sw_high(k=3, weights=queen_1)

    # Calculate neighbor distances using the higher-order spatial weights
    buildings['neighbor_distance_q3'] = momepy.NeighborDistance(buildings, queen_3, "uID", verbose=False).series

    # Create a DataFrame containing the necessary metrics for this city
    city_metrics_df = buildings[["uID", "neighbor_distance_q3"]].copy()

    # Store the DataFrame in the dictionary
    city_metric_dfs[city] = city_metrics_df

    # Display the DataFrame for the current city
    print(f"\nMetrics DataFrame for {city}:")
    print(city_metric_dfs[city].head())

    # Define the output file path using city name and metric
    output_file = os.path.join(output_df_dir, f"{city}_neighbor_distance_Q3.csv")

    # Save the DataFrame as a CSV file
    city_metrics_df.to_csv(output_file, index=False)

    print(f"Saved metrics DataFrame for {city} to {output_file}")


Calculating metrics for Ariel...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/3993208919.py:17: FutureWarning: `momepy.sw_high` is deprecated. Replace it with .higher_order() method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  queen_3 = momepy.sw_high(k=3, weights=queen_1)
/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/3993208919.py:20: FutureWarning: Class based API like `momepy.NeighborDistance` is deprecated. Replace it with `momepy.neighbor_distance` to use functional API instead or pin momepy version <1.0. Class-based API will be removed in 1.0. 
  buildings['neighbor_distance_q3'] = momepy.NeighborDistance(buildings, queen_3, "uID", verbose=False).series



Metrics DataFrame for Ariel:
   uID  neighbor_distance_q3
0    0             65.960086
1    1            223.854714
2    2             96.905410
3    3            115.639879
4    4            164.688516
Saved metrics DataFrame for Ariel to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q3/Ariel_neighbor_distance_Q3.csv
Calculating metrics for Acre, Israel...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/3993208919.py:17: FutureWarning: `momepy.sw_high` is deprecated. Replace it with .higher_order() method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  queen_3 = momepy.sw_high(k=3, weights=queen_1)
/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/3993208919.py:20: FutureWarning: Class based API like `momepy.NeighborDistance` is deprecated. Replace it with `momepy.neighbor_distance` to use functional API instead or pin momepy version <1.0. Class-based API will be removed in 1.0. 
  buildings['neighbor_distance_q3'] = momepy.NeighborDistance(buildings, queen_3, "uID", verbose=False).series



Metrics DataFrame for Acre, Israel:
   uID  neighbor_distance_q3
0    0            131.103288
1    1            208.739001
2    2            127.343615
3    3            118.298655
4    4            175.083139
Saved metrics DataFrame for Acre, Israel to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q3/Acre, Israel_neighbor_distance_Q3.csv
Calculating metrics for Harish...


...


Metrics DataFrame for Tel Aviv:
   uID  neighbor_distance_q3
0    0            124.578091
1    1            201.610202
2    2            235.830444
3    3             92.354157
4    4            151.534608
Saved metrics DataFrame for Tel Aviv to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q3/Tel Aviv_neighbor_distance_Q3.csv
Calculating metrics for Jerusalem...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/3993208919.py:17: FutureWarning: `momepy.sw_high` is deprecated. Replace it with .higher_order() method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  queen_3 = momepy.sw_high(k=3, weights=queen_1)
/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/3993208919.py:20: FutureWarning: Class based API like `momepy.NeighborDistance` is deprecated. Replace it with `momepy.neighbor_distance` to use functional API instead or pin momepy version <1.0. Class-based API will be removed in 1.0. 
  buildings['neighbor_distance_q3'] = momepy.NeighborDistance(buildings, queen_3, "uID", verbose=False).series



Metrics DataFrame for Jerusalem:
   uID  neighbor_distance_q3
0    0            121.909552
1    1            131.074745
2    2            225.911765
3    3             72.594325
4    4            121.442453
Saved metrics DataFrame for Jerusalem to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q3/Jerusalem_neighbor_distance_Q3.csv
Calculating metrics for Beer Sheva...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/3993208919.py:17: FutureWarning: `momepy.sw_high` is deprecated. Replace it with .higher_order() method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  queen_3 = momepy.sw_high(k=3, weights=queen_1)
/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/3993208919.py:20: FutureWarning: Class based API like `momepy.NeighborDistance` is deprecated. Replace it with `momepy.neighbor_distance` to use functional API instead or pin momepy version <1.0. Class-based API will be removed in 1.0. 
  buildings['neighbor_distance_q3'] = momepy.NeighborDistance(buildings, queen_3, "uID", verbose=False).series



Metrics DataFrame for Beer Sheva:
   uID  neighbor_distance_q3
0    0            172.122820
1    1            109.874145
2    2            114.544431
3    3            213.620805
4    4            237.892361
Saved metrics DataFrame for Beer Sheva to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q3/Beer Sheva_neighbor_distance_Q3.csv


shared_walls_length

In [8]:
output_df_dir = os.path.join(output_dir, "shared_walls_length")

# Dictionary to store the DataFrame for each city
city_metric_dfs = {}

# Loop through each city to calculate shared walls length based on stored data
for city in buildings_data.keys():
    print(f"Calculating shared walls length for {city}...")

    # Retrieve the data for the city from the dictionaries
    buildings = buildings_data[city]
    tessellation = tessellation_data[city]

    # Calculate shared walls length for each building
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        buildings["shared_walls_length"] = momepy.SharedWalls(buildings).series

    # Create a DataFrame containing the necessary metrics for this city
    city_metrics_df = buildings[["uID", "shared_walls_length"]].copy()

    # Store the DataFrame in the dictionary
    city_metric_dfs[city] = city_metrics_df

    # Display the DataFrame for the current city
    print(f"\nMetrics DataFrame for {city}:")
    print(city_metric_dfs[city].head())

    # Define the output file path using the city name and metric with '.pkl' extension
    output_file = os.path.join(output_df_dir, f"{city}_shared_walls_length.csv")

    # Save the DataFrame as a pickle file
    city_metrics_df.to_csv(output_file, index=False)


    print(f"Saved metrics DataFrame for {city} as pickle to {output_file}")

Calculating shared walls length for Ariel...

Metrics DataFrame for Ariel:
   uID  shared_walls_length
0    0         1.115247e+01
1    1         0.000000e+00
2    2         2.842171e-14
3    3        -1.421085e-14
4    4         0.000000e+00
Saved metrics DataFrame for Ariel as pickle to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/shared_walls_length/Ariel_shared_walls_length.csv
Calculating shared walls length for Acre, Israel...

Metrics DataFrame for Acre, Israel:
   uID  shared_walls_length
0    0         1.007738e+02
1    1        -1.421085e-14
2    2         4.966819e+01
3    3         3.643347e+01
4    4         4.211078e+01
Saved metrics DataFrame for Acre, Israel as pickle to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/shared_walls_length/Acre, Israel_shared_walls_length.csv
Calculating shared walls length for Harish...

Metrics DataFrame for Harish:
   uID  shared_walls_length
0    0         0.000000e+00
1    1         0.000000e+00
2    2         3.552714e-15
3

Covered Area

In [9]:
output_df_dir = os.path.join(output_dir, "covered_area")

# Dictionary to store the DataFrame for each city
city_metric_dfs = {}

# Loop through each city to calculate metrics based on stored data
for city in buildings_data.keys():
    print(f"Calculating metrics for {city}...")

    # Retrieve the data for the city from the dictionaries
    buildings = buildings_data[city]
    streets = streets_data[city]
    tessellation = tessellation_data[city]  
    
    queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tessellation, ids="uID", silence_warnings=True)
    # Create a higher-order spatial weights matrix for tessellation (considering neighbors up to 3rd order)
    queen_3 = momepy.sw_high(k=3, weights=queen_1)

    # Calculate covered area
    tessellation["covered_area"] = momepy.CoveredArea(tessellation, queen_3, "uID", verbose=False).series

    # Create a DataFrame containing the necessary metrics for this city
    city_metrics_df = tessellation[["uID", "covered_area"]].copy()

    # Store the DataFrame in the dictionary
    city_metric_dfs[city] = city_metrics_df

    # Display the DataFrame for the current city
    print(f"\nMetrics DataFrame for {city}:")
    print(city_metric_dfs[city].head())

    # Define the output file path using city name and metric
    output_file = os.path.join(output_df_dir, f"{city}_covered_area.csv")

    # Save the DataFrame as a CSV file
    city_metrics_df.to_csv(output_file, index=False)

    print(f"Saved metrics DataFrame for {city} to {output_file}")


Calculating metrics for Ariel...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/1476564652.py:17: FutureWarning: `momepy.sw_high` is deprecated. Replace it with .higher_order() method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  queen_3 = momepy.sw_high(k=3, weights=queen_1)
/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/1476564652.py:20: FutureWarning: `momepy.CoveredArea` is deprecated. Replace it with `.describe()` method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  tessellation["covered_area"] = momepy.CoveredArea(tessellation, queen_3, "uID", verbose=False).series



Metrics DataFrame for Ariel:
    uID   covered_area
0  2110   93548.416092
1   326  388090.379057
2  2036  380579.221972
3  1991   93548.416092
4  2037  386815.732962
Saved metrics DataFrame for Ariel to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/covered_area/Ariel_covered_area.csv
Calculating metrics for Acre, Israel...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/1476564652.py:17: FutureWarning: `momepy.sw_high` is deprecated. Replace it with .higher_order() method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  queen_3 = momepy.sw_high(k=3, weights=queen_1)
/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/1476564652.py:20: FutureWarning: `momepy.CoveredArea` is deprecated. Replace it with `.describe()` method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  tessellation["covered_area"] = momepy.CoveredArea(tessellation, queen_3, "uID", verbose=False).series



Metrics DataFrame for Acre, Israel:
   uID   covered_area
0  261  287784.845973
1  239  272959.417674
2  230  212175.848316
3  221  161101.128048
4  224  124649.835641
Saved metrics DataFrame for Acre, Israel to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/covered_area/Acre, Israel_covered_area.csv
Calculating metrics for Harish...

Metrics DataFrame for Harish:
   uID   covered_area
0  497  103914.328968
1  496  109935.628371
2  628   94775.392668
3  629  118227.245304
4  624   98883.055766
Saved metrics DataFrame for Harish to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/covered_area/Harish_covered_area.csv
Calculating metrics for Modiin...


...


Metrics DataFrame for Tel Aviv:
     uID   covered_area
0  10274  107936.049250
1  24334  194187.236140
2  14014  208538.638791
3  13953  192313.669057
4  14010  257055.212661
Saved metrics DataFrame for Tel Aviv to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/covered_area/Tel Aviv_covered_area.csv
Calculating metrics for Jerusalem...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/1476564652.py:17: FutureWarning: `momepy.sw_high` is deprecated. Replace it with .higher_order() method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  queen_3 = momepy.sw_high(k=3, weights=queen_1)
/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/1476564652.py:20: FutureWarning: `momepy.CoveredArea` is deprecated. Replace it with `.describe()` method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  tessellation["covered_area"] = momepy.CoveredArea(tessellation, queen_3, "uID", verbose=False).series



Metrics DataFrame for Jerusalem:
     uID   covered_area
0  32847   87301.661653
1  32755  168869.235674
2  32812   93310.916759
3  32801  139331.804687
4  32798  137390.140844
Saved metrics DataFrame for Jerusalem to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/covered_area/Jerusalem_covered_area.csv
Calculating metrics for Beer Sheva...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/1476564652.py:17: FutureWarning: `momepy.sw_high` is deprecated. Replace it with .higher_order() method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  queen_3 = momepy.sw_high(k=3, weights=queen_1)
/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_70019/1476564652.py:20: FutureWarning: `momepy.CoveredArea` is deprecated. Replace it with `.describe()` method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  tessellation["covered_area"] = momepy.CoveredArea(tessellation, queen_3, "uID", verbose=False).series



Metrics DataFrame for Beer Sheva:
     uID   covered_area
0   2839  125351.156256
1  10410   50440.610980
2  14140   46886.131630
3  14142   48704.595090
4  14143   50822.851002
Saved metrics DataFrame for Beer Sheva to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/covered_area/Beer Sheva_covered_area.csv
